In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os
from collections import Counter
from imblearn.over_sampling import SMOTE

In [ ]:
from feature_selection import forwardFeatureSelection

from NonLinCFA import NonLinCFA
from aux_GenLinCFA import prepare_target_binary

from aux_NonLinCFA import *
import random

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


# NonLinCFA aggregations standardized target, no shuffle, no small aggregations



## Aggregating even if corr <0.85

In [ ]:
# basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio']
basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './NonLinCFA/temp_prec_no_small_aggreg/'
plots_folder = './NonLinCFA/for_plots/no_shuffle/no_small_aggreg/'

for basin in basins:
    selected_colnames_CMI5 = []
    outputs = []

    print('####################' + basin + '####################')
    target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv', window_size = 1)
    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
    output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                            'cyclostationary_mean_tg_1w',
                                                                            'cyclostationary_mean_tg_4w', 
                                                                            'cyclostationary_mean_tg_8w',
                                                                            'cyclostationary_mean_tg_12w', 
                                                                                'cyclostationary_mean_tg_16w',
                                                                                'cyclostationary_mean_tg_24w',
                                                                                'cyclostationary_mean_rr', 
                                                                                'cyclostationary_mean_rr_1w',
                                                                                'cyclostationary_mean_rr_4w', 
                                                                                'cyclostationary_mean_rr_8w',
                                                                                'cyclostationary_mean_rr_12w', 
                                                                                'cyclostationary_mean_rr_16w',
                                                                                'cyclostationary_mean_rr_24w'
                                                                                ],
                                                                          target_df_trainVal, eps=eps,
                                                                          max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                           shuffle=False, no_tiny_aggregations = True)
  
    aggregate_trainVal_string = plots_folder + basin + '_trainVal_aggreg'
    aggregate_test_string = plots_folder + basin + '_test_aggreg'
    aggregate_trainVal.to_csv(aggregate_trainVal_string, index = False)
    aggregate_test.to_csv(aggregate_test_string, index = False)
    outputs.append(output)

    #starting_points.append(starting_point)

    res = {
              "delta" : [], 
              "numSelected" : [], 
              "selectedFeatures" : [] 
          }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames_CMI = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames_CMI)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames_CMI[0:5])

    selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames_CMI[0:5]].columns.values)

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames_CMI[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames_CMI[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames_CMI[0:5]]
            
    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    print('###### Linear Regression ######')

    lin_regr = LinearRegression()

    # CMI best 5
    lin_regr.fit(X_train_validation_CMI5, target_df_trainVal['mean_std'])
    print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal['mean_std']),3))
    print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test['mean_std']),3), "\n")

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)
    log_regr.fit(X_train_validation_CMI5.values, target_df_trainVal)
    print("Train accuracy logregr CMI best 5: ", round(log_regr.score(X_train_validation_CMI5.values, target_df_trainVal),3))
    print("Test accuracy logregr CMI best 5: ", round(log_regr.score(X_test_CMI5.values, target_df_test),3), "\n")
    output_string = plots_folder + basin + '_aggregations.npy'
    sel_col_string = plots_folder + basin + '_chosen_features.npy'
    np.save(sel_col_string, selected_colnames_CMI5)
    np.save(output_string, outputs)

####################Adda####################
Number of features: 92

Number of aggregated features: 8

Working on  cyclostationary_mean_tg
REMOVING SMALL AGGREGATIONS OF 1 OR 2 POINTS...............
Let's start removing aggregations of one single point
New aggregation read: 
['mean_9.849860351582512_45.14986054634262']
One small aggregation found!  mean_9.849860351582512_45.14986054634262
Found  1 neighbours!
Found an adjacent aggregation! The correlation with the small one is of  0.8853417173031289
It is the highest corr found till now, we will merge eventually aggreg  0  with  1
Merging aggreg  0  with  1
Dropped column  cyclostationary_mean_tg 0
New columns names: 
['cyclostationary_mean_tg_1', 'cyclostationary_mean_tg_2', 'cyclostationary_mean_tg_3', 'cyclostationary_mean_tg_4', 'cyclostationary_mean_tg_5', 'cyclostationary_mean_tg_6', 'cyclostationary_mean_tg_7']
Name changed
cyclostationary_mean_tg_0
Name changed
cyclostationary_mean_tg_1
Name changed
cyclostationary_mean_tg_2
Na

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Number of features: 55

Number of aggregated features: 8

Working on  cyclostationary_mean_tg
REMOVING SMALL AGGREGATIONS OF 1 OR 2 POINTS...............
Let's start removing aggregations of one single point
New aggregation read: 
['mean_9.149860354382872_44.94986054697731', 'mean_9.149860354382872_45.04986054665997', 'mean_9.24986035398282_44.94986054697731', 'mean_9.24986035398282_45.04986054665997', 'mean_9.049860354782924_45.04986054665997', 'mean_9.149860354382872_45.14986054634262', 'mean_9.34986035358277_45.14986054634262', 'mean_8.949860355182976_45.04986054665997', 'mean_9.149860354382872_45.24986054602528', 'mean_8.849860355583028_45.04986054665997', 'mean_9.149860354382872_45.349860545707934', 'mean_9.24986035398282_45.24986054602528', 'mean_8.949860355182976_45.14986054634262', 'mean_8.849860355583028_45.14986054634262', 'mean_9.24986035398282_45.349860545707934', 'mean_9.049860354782924_45.14986054634262', 'mean_8.74986035598308_45.04986054665997', 'mean_9.149860354382872_

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)



Number of features: 74

Number of aggregated features: 4

Working on  cyclostationary_mean_tg
REMOVING SMALL AGGREGATIONS OF 1 OR 2 POINTS...............
Let's start removing aggregations of one single point
New aggregation read: 
['mean_10.349860349582258_45.04986054665997', 'mean_10.349860349582258_45.14986054634262', 'mean_10.449860349182208_45.04986054665997', 'mean_10.449860349182208_45.14986054634262', 'mean_10.24986034998231_45.14986054634262', 'mean_10.349860349582258_45.24986054602528', 'mean_10.449860349182208_45.24986054602528', 'mean_10.24986034998231_45.24986054602528', 'mean_10.549860348782158_45.04986054665997', 'mean_10.549860348782158_45.14986054634262', 'mean_10.549860348782158_45.24986054602528', 'mean_10.149860350382362_45.14986054634262', 'mean_10.149860350382362_45.24986054602528', 'mean_10.24986034998231_45.34986054570794', 'mean_10.549860348782158_45.34986054570794', 'mean_10.149860350382362_45.34986054570794', 'mean_10.049860350782412_45.24986054602528', 'mean

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Ticino####################
Number of features: 92

Number of aggregated features: 4

Working on  cyclostationary_mean_tg
REMOVING SMALL AGGREGATIONS OF 1 OR 2 POINTS...............
Let's start removing aggregations of one single point
New aggregation read: 
['mean_8.949860355182974_45.24986054602528', 'mean_8.949860355182974_45.349860545707934', 'mean_9.049860354782922_45.24986054602528', 'mean_9.049860354782922_45.349860545707934', 'mean_8.849860355583024_45.349860545707934', 'mean_8.849860355583024_45.449860545390585', 'mean_8.749860355983076_45.449860545390585', 'mean_8.849860355583024_45.54986054507325', 'mean_8.849860355583024_45.649860544755896', 'mean_8.949860355182974_45.449860545390585', 'mean_8.749860355983076_45.649860544755896', 'mean_8.949860355182974_45.54986054507325', 'mean_8.749860355983076_45.54986054507325', 'mean_8.749860355983076_45.74986054443856', 'mean_8.649860356383128_45.649860544755896', 'mean_8.649860356383128_45.74986054443856', 'mean_8.

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


## Aggregating only if corr > 0.85

In [ ]:
# basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio']
basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './NonLinCFA/temp_prec_no_small_aggreg_if_highcorr/'
plots_folder = './NonLinCFA/for_plots/no_shuffle/no_small_aggreg_if_highcorr/'

for basin in basins:
    selected_colnames_CMI5 = []
    outputs = []

    print('####################' + basin + '####################')
    target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv', window_size = 1)
    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
    output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                            'cyclostationary_mean_tg_1w',
                                                                            'cyclostationary_mean_tg_4w', 
                                                                            'cyclostationary_mean_tg_8w',
                                                                            'cyclostationary_mean_tg_12w', 
                                                                                'cyclostationary_mean_tg_16w',
                                                                                'cyclostationary_mean_tg_24w',
                                                                                'cyclostationary_mean_rr', 
                                                                                'cyclostationary_mean_rr_1w',
                                                                                'cyclostationary_mean_rr_4w', 
                                                                                'cyclostationary_mean_rr_8w',
                                                                                'cyclostationary_mean_rr_12w', 
                                                                                'cyclostationary_mean_rr_16w',
                                                                                'cyclostationary_mean_rr_24w'
                                                                                ],
                                                                          target_df_trainVal, eps=eps,
                                                                          max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                           shuffle=False, no_tiny_aggregations = True, exclude_lowcorr=True)
  
    aggregate_trainVal_string = plots_folder + basin + '_trainVal_aggreg'
    aggregate_test_string = plots_folder + basin + '_test_aggreg'
    aggregate_trainVal.to_csv(aggregate_trainVal_string, index = False)
    aggregate_test.to_csv(aggregate_test_string, index = False)
    outputs.append(output)

    #starting_points.append(starting_point)

    res = {
              "delta" : [], 
              "numSelected" : [], 
              "selectedFeatures" : [] 
          }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames_CMI = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames_CMI)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames_CMI[0:5])

    selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames_CMI[0:5]].columns.values)

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames_CMI[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames_CMI[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames_CMI[0:5]]
            
    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    print('###### Linear Regression ######')

    lin_regr = LinearRegression()

    # CMI best 5
    lin_regr.fit(X_train_validation_CMI5, target_df_trainVal['mean_std'])
    print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal['mean_std']),3))
    print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test['mean_std']),3), "\n")

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)
    log_regr.fit(X_train_validation_CMI5.values, target_df_trainVal)
    print("Train accuracy logregr CMI best 5: ", round(log_regr.score(X_train_validation_CMI5.values, target_df_trainVal),3))
    print("Test accuracy logregr CMI best 5: ", round(log_regr.score(X_test_CMI5.values, target_df_test),3), "\n")
    output_string = plots_folder + basin + '_aggregations.npy'
    sel_col_string = plots_folder + basin + '_chosen_features.npy'
    np.save(sel_col_string, selected_colnames_CMI5)
    np.save(output_string, outputs)

####################Adda####################
Number of features: 92

Number of aggregated features: 8

Working on  cyclostationary_mean_tg
REMOVING SMALL AGGREGATIONS OF 1 OR 2 POINTS...............
Let's start removing aggregations of one single point
One small aggregation found! 
Found  1 neighbours!
Found an adjacent aggregation! The correlation with the small one is of  0.8853417173031289
It is the highest corr found till now, we will merge eventually aggreg  0  with  1
Merging aggreg  0  with  1
Dropped column  cyclostationary_mean_tg 0
Name changed
cyclostationary_mean_tg_0
Name changed
cyclostationary_mean_tg_1
Name changed
cyclostationary_mean_tg_2
Name changed
cyclostationary_mean_tg_3
Name changed
cyclostationary_mean_tg_4
Name changed
cyclostationary_mean_tg_5
Name changed
cyclostationary_mean_tg_6
Updating  cyclostationary_mean_tg _ 0
The new length of output is  7  the last aggreg considered was the  1
The output at this point is: 
['mean_9.849860351582512_45.7498605444385

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Number of features: 55

Number of aggregated features: 8

Working on  cyclostationary_mean_tg
REMOVING SMALL AGGREGATIONS OF 1 OR 2 POINTS...............
Let's start removing aggregations of one single point
No aggregation done.
No aggregation done.
No aggregation done.
No aggregation done.
No aggregation done.
One small aggregation found! 
Found  5 neighbours!
Found an adjacent aggregation! The correlation with the small one is of  0.9907225879182914
It is the highest corr found till now, we will merge eventually aggreg  5  with  0
Merging aggreg  5  with  0
Dropped column  cyclostationary_mean_tg 5
cyclostationary_mean_tg_0
cyclostationary_mean_tg_1
cyclostationary_mean_tg_2
cyclostationary_mean_tg_3
cyclostationary_mean_tg_4
Name changed
cyclostationary_mean_tg_5
Name changed
cyclostationary_mean_tg_6
Updating  cyclostationary_mean_tg _ 0
The new length of output is  7  the last aggreg considered was the  6
The output at this point is: 
['mean_9.149860354382872_44.94986054697731', '

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Number of features: 74

Number of aggregated features: 4

Working on  cyclostationary_mean_tg
REMOVING SMALL AGGREGATIONS OF 1 OR 2 POINTS...............
Let's start removing aggregations of one single point
No aggregation done.
One small aggregation found! 
Found  7 neighbours!
Found an adjacent aggregation! The correlation with the small one is of  0.9695700047132331
It is the highest corr found till now, we will merge eventually aggreg  1  with  0
Merging aggreg  1  with  0
Dropped column  cyclostationary_mean_tg 1
cyclostationary_mean_tg_0
Name changed
cyclostationary_mean_tg_1
Name changed
cyclostationary_mean_tg_2
Updating  cyclostationary_mean_tg _ 0
The new length of output is  3  the last aggreg considered was the  2
The output at this point is: 
['mean_10.349860349582258_45.04986054665997', 'mean_10.349860349582258_45.14986054634262', 'mean_10.449860349182208_45.04986054665997', 'mean_10.449860349182208_45.14986054634262', 'mean_10.24986034998231_45.14986054634262', 'mean_10.

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Number of features: 92

Number of aggregated features: 4

Working on  cyclostationary_mean_tg
REMOVING SMALL AGGREGATIONS OF 1 OR 2 POINTS...............
Let's start removing aggregations of one single point
No aggregation done.
One small aggregation found! 
Found  8 neighbours!
Found an adjacent aggregation! The correlation with the small one is of  0.9843849615285439
It is the highest corr found till now, we will merge eventually aggreg  1  with  0
Found an adjacent aggregation! The correlation with the small one is of  0.9641998371084587
Merging aggreg  1  with  0
Dropped column  cyclostationary_mean_tg 1
cyclostationary_mean_tg_0
Name changed
cyclostationary_mean_tg_1
Name changed
cyclostationary_mean_tg_2
Updating  cyclostationary_mean_tg _ 0
The new length of output is  3  the last aggreg considered was the  2
The output at this point is: 
['mean_8.949860355182974_45.24986054602528', 'mean_8.949860355182974_45.349860545707934', 'mean_9.049860354782922_45.24986054602528', 'mean_9.

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# Multi task scores

In [ ]:
# for binary classification 

from sklearn.metrics import accuracy_score
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())
    
    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

In [ ]:
# for linear regression 

def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LinearRegression()
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())
    
    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        #res = model_ohe.predict(pd.concat([clusterdf_train_withClass.loc[clusterdf_train_withClass[basin]==1], 
        #                                  clusterdf_val_withClass.loc[clusterdf_val_withClass[basin]==1]], axis = 0).values)
        #print(r2_score(pd.concat([targets_df_train[basin], targets_df_val[basin]], axis = 0).values.ravel(), res))
        print(r2_score(targets_df_test[basin].values.ravel(), res))

In [ ]:
### continuous targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_targets = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                     max_test='2020-01-01', path=path_targets+basin+'.csv', 
                                                                                     window_size = 1)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std

In [ ]:
### binary targets
basins = ['Emiliani1','Emiliani2','Garda_Mincio']

path_targets = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                            max_test='2020-01-01', path=path_targets+basin+'.csv', 
                                                                                            threshold = None, nopeaks = False, window_size = 2)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std

In [ ]:
# basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio']
# basins = ['Dora','Piemonte_Sud', 'Piemonte_Nord']
basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']

### CMI best5 features
path_features = './NonLinCFA/temp_prec/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
lin_regr = LinearRegression()
train_val = pd.concat([train_temp,val_temp])
lin_regr.fit(train_val, targets_df_trainVal[basin])
print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(train_val, targets_df_trainVal[basin]),3))
#print("Valid R2 linear regression CMI best 5: ", round(lin_regr.score(X_valid, y_valid),3))

Train R2 linear regression CMI best 5:  0.219


In [ ]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
-0.014517744669723553
Emiliani2
-0.010037458448500791
Garda_Mincio
-0.07339744154500183


In [ ]:
MTL_scores(clust_basins=['Dora','Piemonte_Sud', 'Piemonte_Nord'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
-0.3907479850750135
Piemonte_Sud
-0.22770794082461743
Piemonte_Nord
-0.15350608126875898


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.2822709660617775
Lambro_Olona
0.2530557137348579
Oglio_Iseo
0.26473079356700535
Ticino
0.30167041842497233
